In [ ]:
#Create grids of level-2 NEXRAD data and identify convective cells using tint package
#Identify the time, lat, and lon of convective initiation for each cell
#Largely based off this tutorial: https://notebook.community/openradar/TINT/examples/tint_demo
#More info on tint: https://github.com/openradar/TINT/tree/master
#More info on pyart: https://github.com/ARM-DOE/pyart

In [ ]:
import numpy as np
import tempfile
import os
import shutil
import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks

In [ ]:
#Connect to the repository (https://noaa-nexrad-level2.s3.amazonaws.com/index.html)
#For example, connect to the Indianapolis, IN (KIND) radar on May 3, 2013 from 00:00 GMT to 23:59 GMT
keys = get_nexrad_keys('kind', start='20130503_000000', end='20130503_235959')

In [ ]:
#create a function to retrieve grids
def get_grid(radar):
    grid = pyart.map.grid_from_radars(
        radar, grid_shape=(10, 241, 241),
        grid_limits=((0, 10000), (-200000.0, 200000.0), (-200000.0, 200000.0)),
        fields=['reflectivity'], gridding_algo='map_gates_to_grid', h_factor=1., nb=1., bsp=0., min_radius=200.)
    return grid

In [ ]:
# Create a temporary directory for our grids
tmp_dir = tempfile.mkdtemp()
print('temporary directory:', tmp_dir)
filenames = []

In [ ]:
#Write the radar files to grid files using pyart
#This can take several minutes, as noted by https://notebook.community/openradar/TINT/examples/tint_demo

for num, key in enumerate(keys):
    print('saving grid', num)
    radar = read_nexrad_key(key)
    grid = get_grid(radar)
    name = os.path.join(tmp_dir, 'grid_' + str(num).zfill(3) + '.nc')
    filenames.append(name)
    pyart.io.write_grid(name, grid)
    del radar, grid

In [ ]:
#Read in the grid files using pyart
grids = (pyart.io.read_grid(fn) for fn in filenames)

In [ ]:
#Initialize the cell tracking method and check the parameters
tracks_obj = Cell_tracks()
tracks_obj.params

In [ ]:
#Detect any cell at or above 35 dBZ
tracks_obj.params['FIELD_THRESH'] = 35

#To be isolated, I am going to require a cell may only have up to 4 contiguous pixels with another cell
tracks_obj.params['ISO'] = 4

#Double check the parameters
tracks_obj.params

In [ ]:
#Get the tracks
tracks_obj.get_tracks(grids)

In [ ]:
#create a dataframe
import pandas as pd
df=pd.DataFrame(tracks_obj.tracks)

In [ ]:
#visualize the first few entries
df.head(20)

In [ ]:
#the dataframe sets uid (cell id) as an object; reset this so that we can sort the df by uid
df = df.reset_index()
df

In [ ]:
#uid is also not in integer format; convert to make future sorting easier
df.uid = df.uid.astype('int')

In [ ]:
#sort by cell id ('uid')
df = df.sort_values(by=['uid'])
df

In [ ]:
#remove cell id duplicates to get a list of the first detection of each cell
df = df.drop_duplicates(subset=['uid'], keep='first')

#Only keep the isolated cells; remove cells where isolated = False
df = df[df['isolated']]
df

In [ ]:
#generate a csv file
df.to_csv('3may2013tracks.csv')

In [ ]:
#Delete the temporary directory when you no longer need the files
shutil.rmtree(tmp_dir)
print('temporary directory removed')